# Semantic Kernel

ในตัวอย่างโค้ดนี้ คุณจะใช้ [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI Framework เพื่อสร้างตัวแทนพื้นฐาน

เป้าหมายของตัวอย่างนี้คือการแสดงขั้นตอนที่เราจะใช้ในตัวอย่างโค้ดเพิ่มเติมในภายหลังเมื่อดำเนินการรูปแบบตัวแทนต่างๆ


## นำเข้าแพ็คเกจ Python ที่จำเป็น


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## การสร้าง Client

ในตัวอย่างนี้ เราจะใช้ [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) เพื่อเข้าถึง LLM

`ai_model_id` ถูกกำหนดเป็น `gpt-4o-mini` ลองเปลี่ยนโมเดลเป็นโมเดลอื่นที่มีอยู่ในตลาดของ GitHub Models เพื่อดูผลลัพธ์ที่แตกต่างกัน

เพื่อให้เราใช้ `Azure Inference SDK` ซึ่งใช้สำหรับ `base_url` ของ GitHub Models เราจะใช้ตัวเชื่อมต่อ `OpenAIChatCompletion` ภายใน Semantic Kernel นอกจากนี้ยังมี [ตัวเชื่อมต่ออื่นๆ](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) ที่สามารถใช้ Semantic Kernel กับผู้ให้บริการโมเดลรายอื่นได้


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## การสร้าง Agent

ด้านล่างนี้เราจะสร้าง Agent ที่ชื่อว่า `TravelAgent`

สำหรับตัวอย่างนี้ เราใช้คำแนะนำที่เรียบง่ายมาก คุณสามารถเปลี่ยนคำแนะนำเหล่านี้เพื่อดูว่า Agent ตอบสนองแตกต่างกันอย่างไร


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## การรัน Agent

ตอนนี้เราสามารถรัน Agent ได้โดยการกำหนด thread ที่เป็นประเภท `ChatHistoryAgentThread` ข้อความระบบที่จำเป็นจะถูกส่งให้กับ invoke_stream ของ agent ผ่าน keyword argument `messages`

หลังจากกำหนดสิ่งเหล่านี้แล้ว เราจะสร้าง `user_inputs` ซึ่งจะเป็นสิ่งที่ผู้ใช้ส่งไปยัง agent ในกรณีนี้ เราได้ตั้งข้อความนี้เป็น `Plan me a sunny vacation`

คุณสามารถเปลี่ยนข้อความนี้ได้ตามต้องการเพื่อดูว่า agent จะตอบสนองแตกต่างกันอย่างไร


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้องมากที่สุด แต่โปรดทราบว่าการแปลโดยอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาดั้งเดิมควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษามนุษย์ที่มีความเชี่ยวชาญ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความที่ผิดพลาดซึ่งเกิดจากการใช้การแปลนี้
